In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx to Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx
User uploaded file "Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx" with length 6481955 bytes


In [23]:
import pandas as pd
import numpy as np

#Read dataset
features =  pd.read_excel("Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx")

Rimpiazziamo i codici relativi a determinate feature con i nomi delle feature, così da poter utilizzare la funzione get_dummies() ed ottenere un one-hot-encoding

In [24]:
features["Diploma_scuola_superiore"] = features["Diploma_scuola_superiore"].map({1: "Classico", 2: "Scientifico", 3: "Linguistico", 4: "Magistrale", 5: "Artistico", 6: "Tecnico", 7: "Professionale", 8: "Altro_italiano", 9: "Estero", 99: "Non_disponibile"})
features["area_geografica_scuolasuperiore"] = features["area_geografica_scuolasuperiore"].map({1: "Emilia_romagna", 2: "Nord", 3: "Centro", 4: "Sud_isole", 5: "Estero", 99: "Non_disponibile"})
features["area_geografica_residenza"] = features["area_geografica_residenza"].map({1: "Emilia_romagna", 2: "Nord", 3: "Centro", 4: "Sud_isole", 5: "Estero", 99: "Non_disponibile"})
features["Ambito"] = features["Ambito"].map({1: "Economia", 2: "Farmacia", 3: "Giurisprudenza", 4: "Ingegneria", 5: "Lingue", 6: "Medicina", 7: "Veterinaria", 8: "Psicologia", 9: "Scienze", 10: "Scienze_agroalimanetari", 11: "Scienze_educazione_formazione", 12: "Scienze_motorie", 13: "Scienze_politiche", 14: "Scienze_statistiche", 15: "Sociologia", 16: "Studi_umanistici"})
features = pd.get_dummies(features)

Create and add age in DataFrame

In [25]:
from datetime import datetime, date

def getAge(dataNascita, coorte):
    ts = pd.to_datetime(str(dataNascita)) 
    born = ts.strftime('%d/%m/%Y')
    born = datetime.strptime(born, "%d/%m/%Y").date()
    date = "31/12/"+str(coorte)
    dataAttuale = datetime.strptime(date, "%d/%m/%Y").date()
    age = dataAttuale.year - born.year - ((dataAttuale.month, dataAttuale.day) < (born.month, born.day))
    return age

#add new column with NaN
features["Eta"] = np.nan

for ind in features.index:
    studente = features.iloc[ind]
    dataNascita = studente["DataNascita"]
    coorte = studente["Coorte"]
    #fill the new column with age 
    features.at[ind,'Eta'] = getAge(dataNascita, coorte)

print(features)

       ID_Studente  Coorte DataNascita  Genere  voto_scuola_superiore  \
0                1    2017  1998-02-26       1                   79.0   
1                2    2016  1997-10-06       1                   75.0   
2                3    2017  1986-08-29       1                   74.0   
3                4    2017  1998-01-23       1                   63.0   
4                5    2018  1999-07-04       2                   98.0   
...            ...     ...         ...     ...                    ...   
67275        67276    2016  1997-04-22       1                   83.0   
67276        67277    2017  1998-01-08       1                  100.0   
67277        67278    2016  1997-04-16       2                   75.0   
67278        67279    2017  1995-10-28       2                    NaN   
67279        67280    2016  1992-09-15       1                   65.0   

       Classe_ISEE  Merito_ISEE  OFA_assegnati  OFA_superati   CdS  ...  \
0                5     36332.49              0  

Abbiamo sostituito i valori NaN della feature "voto_scuola_superiore" con la media. Inoltre sono state droppate le colonne non utili ai fini dell'addestramento, ed infine eliminate le tuple contente almeno un NaN

In [26]:
mean_value = features['voto_scuola_superiore'].mean()
features['voto_scuola_superiore'] = features['voto_scuola_superiore'].fillna(mean_value)
features = features.drop('ID_Studente', axis = 1)
features = features.drop('Classe_ISEE', axis = 1)
features = features.drop('DataNascita', axis = 1)
features = features.drop('OFA_superati', axis = 1)
features = features.drop('Coorte', axis = 1)
features = features.dropna()

Abbiamo bilanciato il dataset con approccio undersampling pareggiando il numero di dropout e non droput

In [27]:
#BILANCIAMENTO 1
#Data preprocessing

dropout = pd.DataFrame()
noDropout = pd.DataFrame()

for ind in features.index:
    studente = pd.DataFrame(features.loc[[ind]])
    if features["Abbandoni"][ind] == 1:
        dropout = pd.concat([studente, dropout], ignore_index=True, axis = 0)
    else:
        noDropout = pd.concat([studente, noDropout], ignore_index=True, axis = 0)

#Bilanciamento dataset
final = pd.DataFrame()
for ind in dropout.index:
    studente_drop = pd.DataFrame(dropout.loc[[ind]])
    studente_noDrop = pd.DataFrame(noDropout.loc[[ind]])
    final = pd.concat([studente_drop, final], ignore_index=True, axis = 0)
    final = pd.concat([studente_noDrop, final], ignore_index=True, axis = 0)

features = final
print(len(features))


4186


La seguente cella serve a prescindere dal tipo di bilanciamento 

In [28]:
#Dividing the data into attributes and labels
X = features.drop('Abbandoni', axis=1)
y = features['Abbandoni']

In [29]:
#BILANCIAMENTO 2
"""from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler

#Resampling data
rus = RandomUnderSampler(sampling_strategy = "majority", random_state=42)
X_res, y_res = rus.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))
X = X_res
y = y_res
"""

'from collections import Counter\nfrom sklearn.datasets import make_classification\nfrom imblearn.under_sampling import RandomUnderSampler\n\n#Resampling data\nrus = RandomUnderSampler(sampling_strategy = "majority", random_state=42)\nX_res, y_res = rus.fit_resample(X, y)\nprint(\'Resampled dataset shape %s\' % Counter(y_res))\nX = X_res\ny = y_res\n'

In [30]:
from sklearn.model_selection import train_test_split as tts
#Dividing data into training and test sets
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.20, random_state=42)

In [31]:
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error,r2_score
from sklearn import metrics
model = RandomForestClassifier(n_estimators = 207, random_state = 42)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=207, random_state=42)

In [32]:
predictions = model.predict(X_test)
print(predictions)

print(r2_score(y_test,predictions))
print("Evaluating the Algorithm:\n")
print(metrics.confusion_matrix(y_test,predictions))
print(metrics.classification_report(y_test,predictions))


print("Evaluating Prediction Accuracy:\n")
print("Accuracy:", metrics.accuracy_score(y_test, predictions))

#print(*predictions, sep ="\n")

[1 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 1 1 0 1 0 1 1 0 1 1 1 0 0 1 1 0 1 1 0 0 0
 0 0 1 0 0 0 1 0 0 0 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 1 0 0 0 1 1 0 1 0 0 1
 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 0 0 1 1 0 1 1 1 0 1 1 0 0 0 0 1 0 1 1
 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 1 1 0 1 0 1 0 1 1 1 0 1
 0 1 0 1 0 1 0 1 0 0 1 1 1 0 0 0 1 1 1 0 1 0 1 1 1 0 1 1 1 0 0 1 0 0 1 0 0
 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 1 0 0 1 1 0 0 0 0
 1 0 0 1 0 1 1 0 0 1 0 0 1 1 1 1 1 0 1 1 1 0 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1
 0 1 1 0 0 1 1 1 0 0 0 1 1 0 0 1 1 1 1 0 1 1 0 1 0 0 0 1 1 0 0 1 1 1 0 0 0
 0 0 0 1 1 0 1 1 0 1 0 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 0 1 1 0
 1 0 0 0 1 0 1 1 1 0 1 1 0 0 1 1 0 1 0 1 0 1 0 0 1 0 0 0 0 0 1 1 1 1 0 1 1
 1 0 0 0 1 1 1 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0 0
 1 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1 1 1
 1 1 0 0 0 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 0 0 1 1 0
 0 0 0 0 1 0 0 0 0 0 1 1 